# Dataloader与Dataset

## 1. 学习内容
本节第一部分介绍pytorch的数据读取机制，通过一个人民币分类实验来学习pytorch是如何从硬盘中读取数据的，并且深入学习数据读取过程中涉及到的两个模块Dataloader与Dataset

---

## 2. 人民币二分类

任务: 输入1元或100元纸币的图像，判断它是1或者100？    
解决方案: 将输入的图像看成因变量$x$，找到一个模型$f(x)$，完成对输出结果0/1的映射
<img src="picture/人民币二分类.png" width="500">    
### 2.1 数据模块
根据之前的机器学习训练模型的框图结构(五大模块)，今天首先介绍数据模块:    
<img src="picture/数据分类框图.png" width="600">  

#### a) 数据收集: 

收集原始图片和图片Label

---

#### b) 数据划分: 
将数据分为三部分: Train, Valid and Test  

**训练数据集（Training Set):**

是一些我们已经知道输入和输出的数据集训练机器去学习，通过拟合去寻找模型的初始参数。例如在神经网络（Neural Networks)中， 我们用训练数据集和反向传播算法（Backpropagation）去每个神经元找到最优的比重（Weights)。

**验证数据集（Validation Set):**

也是一些我们已经知道输入和输出的数据集，通过让机器学习去优化调整模型的参数，在神经网络中， 我们用验证数据集去寻找最优的网络深度（number of hidden layers)，或者决定反向传播算法的停止点；在普通的机器学习中常用的交叉验证（Cross Validation) 就是把训练数据集本身再细分成不同的验证数据集去训练模型。

**测试数据集（Test Set):**

用户测试模型表现的数据集，根据误差（一般为预测输出与实际输出的不同）来判断一个模型的好坏。



**为什么验证数据集和测试数据集两者都需要？**

因为验证数据集（Validation Set)用来调整模型参数从而选择最优模型，模型本身已经同时知道了输入和输出，所以从验证数据集上得出的误差（Error)会有偏差（Bias)。

但是我们只用测试数据集(Test Set) 去评估模型的表现，并不会去调整优化模型。

在传统的机器学习中，这三者一般的比例为training/validation/test = 50/25/25, 但是有些时候如果模型不需要很多调整只要拟合就可时，或者training本身就是training+validation (比如cross validation)时，也可以training/test =7/3.

但是在深度学习中，由于数据量本身很大，而且训练神经网络需要的数据很多，可以把更多的数据分给training，而相应减少validation和test。

在我的论文中，我将采用训练集(cross validation)去训练和选择超参数

参考[知乎DataQuestion](https://zhuanlan.zhihu.com/p/29133576)

---
#### c) 数据读取
pytorch数据读取的核心是DataLoader，其还细分为Sampler和DataSet:
- Sampler: 作用是生成Index，也就是样本的序号
- DataSet: 根据索引去读取图片和标签

---

#### d) 数据预处理
在数据读取之后，还需要数据的中心化，标准化，旋转，裁剪等操作，这个在pytorch中是通过transforms实现

### 2.2 DataLoader
#### 2.2.1 torch.utils.data.DataLoader()    
功能: 构建可迭代的数据装载器, 每次读入一个batchsize的数据
- dataset:    Dataset类，决定数据从哪读取及如何读取
- batchsize:  批大小
- num_works:  是否多进程读取数据
- shuffle:    每个epoch是否乱序
- drop_last:  当样本数不能被batchsize整除时，是否舍弃最后一批数据

```python
DataLoader( dataset, 
            batch_size=1,
            shuffle=False,
            sampler=None, 
            batch_sampler=None, 
            num_workers=0, 
            collate_fn=None, 
            pin_memory=False, 
            drop_last=False,
            timeout=0, 
            worker_init_fn=None, 
            multiprocessing_context=None)
```

#### 2.2.2 Epoch, Iteration and batchsize的关系
- Epoch: 所有训练样本都已输入到模型中，称为一个Epoch 
- Iteration:一批样本输入到模型中，称之为一个Iteration 
- Batchsize:批大小，决定一个Epoch有多少个Iteration 


例子:
假设有样本总数:80, Batchsize:8    
一次Epoch中总共有10次Iter: 1 Epoch = 10 Iteration    
假设样本总数:87, Batchsize:8，不能整除      
1 Epoch = (drop_last == True) ? 10 : 11    

#### 2.2.3 torch.utils.data.Dataset()
```python
class Dataset(object):
    def __getitem__(self, index):
        raise NotImplementedError
        
    def __add__(self, other):
        return ConcatDataset([self, other])
```
 
功能:Dataset抽象类，所有自定义的customer Dataset需要继承它，并且复写 __getitem__()
- <font color=blue>getitem</font>: 是Dataset的关键方法，它完成了:接受一个索引，并返回一个样本

---

### 2.3 学习人民币二分类中的数据模块

弄清楚下面三个问题，就是**数据读取**的关键:
- 1. 读哪些数据？
- 2. 从哪里读数据？
- 3. 怎么读数据？


In [ ]:
# ============================ step 1/5 数据 ============================
# ".."是退回到上级目录; 这三行代码是设置硬盘中的路径，告诉我们从哪里读数据
split_dir = os.path.join("..", "..", "data", "rmb_split")
train_dir = os.path.join(split_dir, "train")
valid_dir = os.path.join(split_dir, "valid")

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

# 构建MyDataset实例
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)

# 构建DataLoder
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)

#### 2.3.1 自己重写Dataset

In [ ]:
class RMBDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        rmb面额分类任务的Dataset
        :param data_dir: str, 数据集所在路径
        :param transform: torch.transform，数据预处理
        """
        self.label_name = {"1": 0, "100": 1}
        self.data_info = self.get_img_info(data_dir)  # data_info存储所有图片路径和标签，在DataLoader中通过index读取样本
        self.transform = transform
    
    # 根据一个index，读取一个图片 和 对应的label
    def __getitem__(self, index):
        path_img, label = self.data_info[index]
        img = Image.open(path_img).convert('RGB')     # 0~255

        if self.transform is not None:
            img = self.transform(img)   # 在这里做transform，转为tensor等等

        return img, label

    def __len__(self):
        return len(self.data_info)

    @staticmethod
    # data_info是一个元祖list()，包含所有数据的 path_img 和 label
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            # 遍历类别
            for sub_dir in dirs:
                img_names = os.listdir(os.path.join(root, sub_dir))
                img_names = list(filter(lambda x: x.endswith('.jpg'), img_names))

                # 遍历图片
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    path_img = os.path.join(root, sub_dir, img_name)
                    label = rmb_label[sub_dir]
                    data_info.append((path_img, int(label)))

        return data_info

#### 2.3.2 单步调试DataLoader
```python
for i, data in enumerate(train_loader):# 在此设置断点，观察每次调用train_loader会发生什么

    # forward
    inputs, labels = data
    outputs = net(inputs)
```

1. 从`train_loader`会进入到DataLoader类的`__iter__(self)`方法中


2. `DataLoader.__iter__(self)`中包含二中读取模式：单进程和多进程，其中选取单进程以详细介绍


3. 选取单进程后跳转到，`DataLoader.__next__()`函数，目的是获取index，通过index得到data，其中包含
    - `index=self.__next__index()`, 通过`sampler`得到index
    - `data=self.dataset.fetcher.fetch(index)`, 通过上个函数得到的index得到整合到一个batch的data
    
    
4. 跳转到`dataset.fetcher.fetch(self, possiblely_batched_index)`中，会调用之前我们自己定义的`Dataset.__getitem()__`得到单个数据，再append成一个data list，最后`return self.collate_fo(data)`，整合成一个长度为batch_size的Tensor元祖，第一个是数据，第二个是Label

--- 
<img src="picture/DataLoader框图.png" width="600">  
最后结合一张图回答数据读取的三个问题:    
1. 读哪些: Sampler类告诉我们每次的index
2. 从哪读: 自己定义的Dataset类中会明确数据在硬盘中的位置
3. 怎么读: 通过`Dataset.__getitem()__`得到data 和 label